# Import libraries

In [ ]:
!pip install bitsandbytes>=0.39.0 accelerate>=0.20.0
!pip install transformers

In [ ]:
import inspect
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration

# Instantiate model

In [ ]:
# Instantiate BLIP-2 model. via
# https://huggingface.co/Salesforce/blip2-flan-t5-xxl
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xxl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xxl", device_map="auto", load_in_8bit=True)

# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create directory to store inferences

In [ ]:
os.makedirs('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions', exist_ok=True)

# Import datasets

## US-Politics

In [ ]:
constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constraint22_dataset_uspolitics_test_captioned_Qwen-VL-Chat.csv')
constraint22_dataset_uspolitics_test_captioned_InternLM_XComposer = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constraint22_dataset_uspolitics_test_captioned_InternLM-XComposer.csv')
constraint22_dataset_uspolitics_test_captioned_llava = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constraint22_dataset_uspolitics_test_captioned_llava-v1.5-13b.csv')
constraint22_dataset_uspolitics_test_captioned_BLIP_2 = pd.read_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/constraint22_dataset_uspolitics_test_captioned_BLIP-2.csv')

In [ ]:
constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat = constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat.dropna().reset_index(drop=True)
constraint22_dataset_uspolitics_test_captioned_InternLM_XComposer = constraint22_dataset_uspolitics_test_captioned_InternLM_XComposer.dropna().reset_index(drop=True)
constraint22_dataset_uspolitics_test_captioned_llava = constraint22_dataset_uspolitics_test_captioned_llava.dropna().reset_index(drop=True)
constraint22_dataset_uspolitics_test_captioned_BLIP_2 = constraint22_dataset_uspolitics_test_captioned_BLIP_2.dropna().reset_index(drop=True)

# Set prompts and define a function to call the model

In [ ]:
def prompt_vanilla(entity):
    return inspect.cleandoc(f"""
    What is the role of {entity} in this meme?
    hero: presented in a positive light.
    villain: portrayed negatively, e.g., in an association with adverse traits like wickedness, cruelty, hypocrisy, etc.
    victim: portrayed as suffering the negative impact of someone else’s actions.
    other: not a hero, a villain, or a victim.
    Constraint: Without using any other words, answer either hero, villain, victim, other.""")

In [ ]:
def prompt_with_OCR(entity, OCR):
    return inspect.cleandoc(f"""
    Text on this meme: \"\"\"
    {OCR}
    \"\"\"
    What is the role of {entity} in this meme?
    hero: presented in a positive light.
    villain: portrayed negatively, e.g., in an association with adverse traits like wickedness, cruelty, hypocrisy, etc.
    victim: portrayed as suffering the negative impact of someone else’s actions.
    other: not a hero, a villain, or a victim.
    Constraint: Without using any other words, answer either hero, villain, victim, other.""")

In [ ]:
def prompt_with_caption(entity, caption):
    return inspect.cleandoc(f"""
    Description of this meme: \"\"\"
    {caption}
    \"\"\"
    What is the role of {entity} in this meme?
    hero: presented in a positive light.
    villain: portrayed negatively, e.g., in an association with adverse traits like wickedness, cruelty, hypocrisy, etc.
    victim: portrayed as suffering the negative impact of someone else’s actions.
    other: not a hero, a villain, or a victim.
    Constraint: Without using any other words, answer either hero, villain, victim, other.""")

In [ ]:
# Use BLIP-2 for the inference. via
# https://huggingface.co/Salesforce/blip2-flan-t5-xxl
def get_prediction(image, prompt):
    raw_image = Image.open(image).convert('RGB')
    inputs = processor(raw_image, prompt, return_tensors="pt").to("cuda")
    out = model.generate(**inputs, max_new_tokens=30)
    return processor.decode(out[0], skip_special_tokens=True)

# Define function to clean responses

In [ ]:
def remap(x):
    x = x.lower()
    if x in {'hero', 'villain', 'victim', 'other'}:
        return x
    elif 'hero' in x and 'villain' not in x and 'victim' not in x and 'other' not in x and 'not hero' not in x and 'not a hero' not in x:
        return 'hero'
    elif 'villain' in x and 'hero' not in x and 'victim' not in x and 'other' not in x and 'not villain' not in x and 'not a villain' not in x:
        return 'villain'
    elif 'victim' in x and 'villain' not in x and 'hero' not in x and 'other' not in x and 'not victim' not in x and 'not a victim' not in x:
        return 'victim'
    elif 'other' in x and 'villain' not in x and 'victim' not in x and 'hero' not in x and 'not other' not in x and 'not an other' not in x:
        return 'other'
    else:
        return None

# Call the `get_prediction` function and save inferences

GROUNDING [ABSENT] & PROMPT [VANILLA]

In [ ]:
uspolitics_test_grounding_absent_prompt_vanilla = constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat.copy(deep=True)
uspolitics_test_grounding_absent_prompt_vanilla_images = uspolitics_test_grounding_absent_prompt_vanilla['image'].values
uspolitics_test_grounding_absent_prompt_vanilla_entities = uspolitics_test_grounding_absent_prompt_vanilla['entity'].values
uspolitics_test_grounding_absent_prompt_vanilla['prediction'] = [get_prediction(image, prompt_vanilla(entity)) for image, entity in zip(uspolitics_test_grounding_absent_prompt_vanilla_images, uspolitics_test_grounding_absent_prompt_vanilla_entities)]
uspolitics_test_grounding_absent_prompt_vanilla.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_prompt[VANILLA]_prediction[BLIP-2].csv', index=False)
uspolitics_test_grounding_absent_prompt_vanilla['prediction'] = uspolitics_test_grounding_absent_prompt_vanilla['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_vanilla['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_vanilla['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_vanilla['prediction'] = uspolitics_test_grounding_absent_prompt_vanilla['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_vanilla['role'].values, uspolitics_test_grounding_absent_prompt_vanilla['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_vanilla['role'].values, uspolitics_test_grounding_absent_prompt_vanilla['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

other      605
hero       324
villain     52
victim      14
Name: prediction, dtype: int64
0
0.2823330716605092
              precision    recall  f1-score   support

        hero       0.43      0.57      0.49       246
     villain       0.48      0.10      0.17       250
      victim       0.86      0.05      0.09       249
       other       0.27      0.65      0.38       250

    accuracy                           0.34       995
   macro avg       0.51      0.34      0.28       995
weighted avg       0.51      0.34      0.28       995



GROUNDING [ABSENT] & PROMPT [OCR]

In [ ]:
uspolitics_test_grounding_absent_prompt_with_OCR = constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat.copy(deep=True)
uspolitics_test_grounding_absent_prompt_with_OCR_images = uspolitics_test_grounding_absent_prompt_with_OCR['image'].values
uspolitics_test_grounding_absent_prompt_with_OCR_entities = uspolitics_test_grounding_absent_prompt_with_OCR['entity'].values
uspolitics_test_grounding_absent_prompt_with_OCR_texts = uspolitics_test_grounding_absent_prompt_with_OCR['OCR'].values
uspolitics_test_grounding_absent_prompt_with_OCR['prediction'] = [get_prediction(image, prompt_with_OCR(entity, text)) for image, entity, text in zip(uspolitics_test_grounding_absent_prompt_with_OCR_images, uspolitics_test_grounding_absent_prompt_with_OCR_entities, uspolitics_test_grounding_absent_prompt_with_OCR_texts)]
uspolitics_test_grounding_absent_prompt_with_OCR.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_prompt[OCR]_prediction[BLIP-2].csv', index=False)
uspolitics_test_grounding_absent_prompt_with_OCR['prediction'] = uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_with_OCR['prediction'] = uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_with_OCR['role'].values, uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_with_OCR['role'].values, uspolitics_test_grounding_absent_prompt_with_OCR['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

other      368
hero       357
villain    201
victim      69
Name: prediction, dtype: int64
0
0.43668966219215544
              precision    recall  f1-score   support

        hero       0.48      0.70      0.57       246
     villain       0.45      0.36      0.40       250
      victim       0.83      0.23      0.36       249
       other       0.35      0.51      0.41       250

    accuracy                           0.45       995
   macro avg       0.53      0.45      0.44       995
weighted avg       0.53      0.45      0.44       995



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [Qwen_VL_Chat]

In [ ]:
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat = constraint22_dataset_uspolitics_test_captioned_Qwen_VL_Chat.copy(deep=True)
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images = uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['image'].values
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities = uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['entity'].values
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions = uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['caption'].values
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_images, uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_entities, uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat_captions)]
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_caption[Qwen-VL-Chat]_prompt[CAPTION]_prediction[BLIP-2].csv', index=False)
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_Qwen_VL_Chat['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

hero       355
other      289
villain    244
victim     107
Name: prediction, dtype: int64
0
0.5217601297100533
              precision    recall  f1-score   support

        hero       0.53      0.76      0.62       246
     villain       0.52      0.51      0.51       250
      victim       0.79      0.34      0.47       249
       other       0.45      0.52      0.48       250

    accuracy                           0.53       995
   macro avg       0.57      0.53      0.52       995
weighted avg       0.57      0.53      0.52       995



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [InternLM_XComposer]

In [ ]:
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer = constraint22_dataset_uspolitics_test_captioned_InternLM_XComposer.copy(deep=True)
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_images = uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['image'].values
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_entities = uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['entity'].values
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_captions = uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['caption'].values
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_images, uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_entities, uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer_captions)]
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_caption[InternLM-XComposer]_prompt[CAPTION]_prediction[BLIP-2].csv', index=False)
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_InternLM_XComposer['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

Token indices sequence length is longer than the specified maximum sequence length for this model (759 > 512). Running this sequence through the model will result in indexing errors


hero       378
other      296
villain    227
victim      94
Name: prediction, dtype: int64
0
0.4887189912940779
              precision    recall  f1-score   support

        hero       0.48      0.74      0.58       246
     villain       0.51      0.46      0.48       250
      victim       0.79      0.30      0.43       249
       other       0.42      0.50      0.46       250

    accuracy                           0.50       995
   macro avg       0.55      0.50      0.49       995
weighted avg       0.55      0.50      0.49       995



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [llava]

In [ ]:
uspolitics_test_grounding_absent_prompt_with_caption_llava = constraint22_dataset_uspolitics_test_captioned_llava.copy(deep=True)
uspolitics_test_grounding_absent_prompt_with_caption_llava_images = uspolitics_test_grounding_absent_prompt_with_caption_llava['image'].values
uspolitics_test_grounding_absent_prompt_with_caption_llava_entities = uspolitics_test_grounding_absent_prompt_with_caption_llava['entity'].values
uspolitics_test_grounding_absent_prompt_with_caption_llava_captions = uspolitics_test_grounding_absent_prompt_with_caption_llava['caption'].values
uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(uspolitics_test_grounding_absent_prompt_with_caption_llava_images, uspolitics_test_grounding_absent_prompt_with_caption_llava_entities, uspolitics_test_grounding_absent_prompt_with_caption_llava_captions)]
uspolitics_test_grounding_absent_prompt_with_caption_llava.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_caption[llava-v1.5-13b]_prompt[CAPTION]_prediction[BLIP-2].csv', index=False)
uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_with_caption_llava['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_with_caption_llava['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_llava['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

hero       351
other      301
villain    246
victim      97
Name: prediction, dtype: int64
0
0.4641399545654457
              precision    recall  f1-score   support

        hero       0.46      0.66      0.54       246
     villain       0.50      0.49      0.49       250
      victim       0.77      0.30      0.43       249
       other       0.36      0.43      0.39       250

    accuracy                           0.47       995
   macro avg       0.52      0.47      0.46       995
weighted avg       0.52      0.47      0.46       995



GROUNDING [ABSENT] & PROMPT [CAPTION] & CAPTION [BLIP-2]

In [ ]:
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2 = constraint22_dataset_uspolitics_test_captioned_BLIP_2.copy(deep=True)
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_images = uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['image'].values
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_entities = uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['entity'].values
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_captions = uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['caption'].values
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = [get_prediction(image, prompt_with_caption(entity, caption)) for image, entity, caption in zip(uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_images, uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_entities, uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2_captions)]
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2.to_csv('/content/drive/MyDrive/stance_detection_datasets/inferences/predictions/constraint22_dataset_uspolitics_test_grounding[ABSENT]_caption[BLIP_2]_prompt[CAPTION]_prediction[BLIP-2].csv', index=False)
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: remap(x))
print(uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].value_counts())
print(uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].isna().sum())
uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'] = uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].apply(lambda x: x if x is not None else np.random.choice(['hero', 'villain', 'victim', 'other']))
print(f1_score(uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['hero', 'villain', 'victim', 'other'], average='macro'))
print(classification_report(uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['role'].values, uspolitics_test_grounding_absent_prompt_with_caption_BLIP_2['prediction'].values, labels=['hero', 'villain', 'victim', 'other']))

other      483
hero       311
villain    140
victim      61
Name: prediction, dtype: int64
0
0.3910664712952961
              precision    recall  f1-score   support

        hero       0.46      0.58      0.51       246
     villain       0.44      0.25      0.32       250
      victim       0.84      0.20      0.33       249
       other       0.31      0.59      0.40       250

    accuracy                           0.41       995
   macro avg       0.51      0.41      0.39       995
weighted avg       0.51      0.41      0.39       995

